In [ ]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.colors import ListedColormap

### You need to get your own license file from Gurobi's website ! It is for single machine use only ! 

In [ ]:
import os
import gurobipy as gp

# Set the GRB_LICENSE_FILE environment variable to the correct license file path
os.environ["GRB_LICENSE_FILE"] = "/Users/emrekuru/Developer/Production_Planning/gurobi.lic"

In [ ]:
model = gp.Model("PaintingProcessOptimization")

In [ ]:
# Parameters
num_parts = 60     # Number of parts
demand = {p: 4 for p in range(1, num_parts + 1)}  


parts_colors = {}
colors = ['Red', 'Blue', 'Green', 'Yellow', 'Black', 'Purple', 'Orange']
for p in range(1, num_parts + 1):
    parts_colors[p] = random.choice(colors)

color_cost = {
    'Red': 5,
    'Blue': 6,
    'Green': 7,
    'Yellow': 8,
    'Black': 9,
    'Purple': 10,
    'Orange': 11
}

unit_production_time = {p: 1 for p in range(1, num_parts + 1)}  

In [ ]:
# Define continuous variables for start and end times of production for each part
start_times = model.addVars(range(1, num_parts + 1), vtype=GRB.CONTINUOUS, name="start_times")
end_times = model.addVars(range(1, num_parts + 1), vtype=GRB.CONTINUOUS, name="end_times")

# Define binary variable to indicate order of parts
order = model.addVars(range(1, num_parts + 1), range(1, num_parts + 1), vtype=GRB.BINARY, name="order")
succesor = model.addVars(range(1, num_parts + 1), range(1, num_parts + 1), vtype=GRB.BINARY, name="succesor")
color_change = model.addVars(range(1, num_parts + 1), range(1, num_parts + 1), vtype=GRB.BINARY, name="color_change")

In [ ]:
model.setObjective(
    quicksum(
        color_cost[parts_colors[p]] * color_change[p, q] for p in range(1, num_parts + 1) for q in range(1, num_parts + 1)
    ),
    GRB.MINIMIZE
)

In [ ]:
for p in range(1, num_parts + 1):
    for p_prime in range(1, num_parts + 1):
        if p != p_prime:
            order[p, p_prime].vtype = GRB.BINARY
            
# Ensure each part has at most one immediate successor, with exactly one part having no successor (the end part)
for p in range(1, num_parts + 1):
    model.addConstr(
        quicksum(succesor[p, q] for q in range(1, num_parts + 1) if q != p) <= 1,
        name=f"Single_Successor_{p}"
    )

# Ensure each part has at most one immediate predecessor, with exactly one part having no predecessor (the start part)
for q in range(1, num_parts + 1):
    model.addConstr(
        quicksum(succesor[p, q] for p in range(1, num_parts + 1) if p != q) <= 1,
        name=f"Single_Predecessor_{q}"
    )

# Ensure that there is exactly one part without a successor (the end part)
model.addConstr(
    quicksum(quicksum(succesor[p, q] for q in range(1, num_parts + 1) if q != p) for p in range(1, num_parts + 1)) == num_parts - 1,
    name="Total_Successors"
)


# Ensure mutual exclusivity in ordering between each pair of parts
for p in range(1, num_parts + 1):
    for p_prime in range(1, num_parts + 1):
        if p != p_prime:
            model.addConstr(
                order[p, p_prime] + order[p_prime, p] == 1,
                name=f"Order_Binary_{p}_{p_prime}"
            )

# Link the immediate successor relationship to the order variable
# If part p is the immediate successor of part p_prime, then p_prime should come before p in the order
for p in range(1, num_parts + 1):
    for p_prime in range(1, num_parts + 1):
        if p != p_prime:
            model.addConstr(
                order[p_prime, p] >= succesor[p, p_prime],
                name=f"Order_Successor_Link_{p_prime}_{p}"
            )

# Ensure color change costs are applied when there is a change in color between successive parts
for p in range(1, num_parts + 1):
    for p_prime in range(1, num_parts + 1):
        if p != p_prime and parts_colors[p] != parts_colors[p_prime]:
            model.addConstr(
                color_change[p, p_prime] >= succesor[p, p_prime],
                name=f"Color_Change_{p}_{p_prime}"
            )

# Demand fulfillment constraints to ensure each part meets its demand
for p in range(1, num_parts + 1):
    model.addConstr(
        (end_times[p] - start_times[p]) * unit_production_time[p] >= demand[p],
        name=f"Demand_Fulfillment_{p}"
    )

# No overlap constraints based on the order of parts
# If part p is ordered before part p_prime, then end_times[p] <= start_times[p_prime]
for p in range(1, num_parts + 1):
    for p_prime in range(1, num_parts + 1):
        if p != p_prime:
            model.addConstr(
                end_times[p] <= start_times[p_prime] + (1 - order[p, p_prime]) * 1e6,
                name=f"No_Overlap_{p}_{p_prime}"
            )


In [ ]:
# Set model parameters if necessary, e.g., setting the feasibility tolerance
model.setParam("IntFeasTol", 1e-9)

# Optimize the model
model.optimize()

  7232  6703   13.00008  259   56          -    0.00000      -  81.8   10s
  7288  6732    0.00000   18  278          -    0.00000      -  11.4   15s
  8400  7403   21.00012   80  217          -    0.00000      -  29.5   20s
 10858  8695   49.00000  211  227          -    0.00000      -  40.5   25s
 14126 10286   38.00038  471  284          -    0.00000      -  47.7   30s
 16618 11228    0.00013   67  330          -    0.00000      -  50.8   35s
 19628 12811    0.00046  364  305          -    0.00000      -  52.7   40s
 22693 14772   51.00000  581  374          -    0.00000      -  50.4   45s
 25568 16579   75.00000  888  270          -    0.00000      -  48.1   50s
 28539 18282  108.00000 1242  331          -    0.00000      -  46.2   56s
 29266 19104  108.00000 1325  276          -    0.00000      -  46.0   67s
 31296 20972    0.00014  223  345          -    0.00000      -  44.9   70s
 34333 24226 infeasible  516               -    0.00000      -  42.8   76s
 38808 27958   68.00007 1

In [ ]:
# Check if the model is infeasible
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    model.write("model.ilp")
    
    # Print the constraints that are part of the IIS
    print("\nThe following constraints are part of the IIS:")
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"{c.constrName}")

In [ ]:
start_times

In [ ]:
end_times

In [ ]:
order

In [ ]:
succesor

In [ ]:
color_change

In [ ]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    
    # Extract start and end times for each part
    start_times_values = [start_times[p].X for p in range(1, num_parts + 1)]
    end_times_values = [end_times[p].X for p in range(1, num_parts + 1)]
    
    # Plotting the Gantt chart
    plt.figure(figsize=(20, 6))
    
    # Define color mapping for the parts
    color_map = {
        'Red': 'red',
        'Blue': 'blue',
        'Green': 'green',
        'Yellow': 'yellow',
        'Black': 'black'
    }

    for p in range(1, num_parts + 1):
        plt.barh(p, end_times_values[p-1] - start_times_values[p-1], left=start_times_values[p-1],
                 color=color_map[parts_colors[p]], edgecolor='black', label=parts_colors[p] if p == 1 else "")

    plt.title("Optimal Production Schedule", fontsize=16)
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Parts", fontsize=14)
    plt.xticks(np.arange(0, max(end_times_values) + 1, 1))  # Set x-ticks according to time
    plt.yticks(range(1, num_parts + 1), fontsize=12)  # Set y-ticks for parts
    plt.grid(axis='x', linestyle='--', alpha=0.7)  # Optional grid for better visibility
    plt.xlim(0, max(end_times_values) + 1)  # Set x-axis limits to provide some padding
    plt.tight_layout()  # Adjust layout to prevent clipping of labels

    plt.show()
    
    # Display objective function result
    minimized_cost = model.ObjVal
    print(f"Minimized Total Cost (Objective Value): {minimized_cost}")

else:
    print("No optimal solution found.")